## MaskデータをCOCOフォーマットに変換して学習用のデータセットを作成する
[参考:COCO Formatの作り方](https://qiita.com/harmegiddo/items/da131ae5bcddbbbde41f)

## フォーマット
データ全体のフォーマットは以下の通り  
今回必要そうなのは、  
- Info
- Images
- Annotations
```JSON
{
    "info": {...},
    "licenses": [...],
    "images": [...],
    "annotations": [...],
    "categories": [...], <-- Not in Captions annotations
    "segment_info": [...] <-- Only in Panoptic annotations
}

```

## Info部分の作成
あってもなくてもいいがデータセットの内容について記述するために用意する

### フォーマット
```JSON
"info": {
    "description": "COCO 2017 Dataset",
    "url": "http://cocodataset.org",
    "version": "1.0",
    "year": 2017,
    "contributor": "COCO Consortium",
    "date_created": "2017/09/01"
}
```

In [2]:
# info部分の作成
import json
import collections as cl

def info():
    tmp = cl.OrderedDict() # OrderedDict:値を追加した順序を記憶することができる辞書型風のデータ構造を提供
    tmp["descripion"] = "Predicting_Solar_Flare"
    tmp["version"] = "0.1"
    tmp["year"] = 2020
    tmp["contributor"] = "Akito Komatsu"
    tmp["data_created"] = "2020/09/01"
    return tmp
    
def main():
    query_list = ["info","images","annotations"]
    js = cl.OrderedDict()
    for i in range (len(query_list)):
        tmp = ""
        if query_list[i] == "info":
            tmp =info()
        
        js[query_list[i]] = tmp
    fw = open("datasets.json","w")
    json.dump(js,fw,indent=2)

if __name__ == "__main__":
    main()



In [3]:
dataset_obj = open("datasets.json","r")
dataset_json =json.load(dataset_obj)
dataset_json

{'info': {'descripion': 'Predicting_Solar_Flare',
  'version': '0.1',
  'year': 2020,
  'contributor': 'Akito Komatsu',
  'data_created': '2020/09/01'},
 'images': '',
 'annotations': ''}

## imagesの作成
IDはユニークで後のAnnotation等に関連付けられる  
→time？検討
フォーマット
```JSON
"images": [
    {
        "license": 4,
        "file_name": "000000397133.jpg",
        "coco_url": "http://images.cocodataset.org/val2017/000000397133.jpg",
        "height": 427,
        "width": 640,
        "date_captured": "2013-11-14 17:02:52",
        "flickr_url": "http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg",
        "id": 397133
    },
    {
        "license": 1,
        "file_name": "000000037777.jpg",
        "coco_url": "http://images.cocodataset.org/val2017/000000037777.jpg",
        "height": 230,
        "width": 352,
        "date_captured": "2013-11-14 20:55:31",
        "flickr_url": "http://farm9.staticflickr.com/8429/7839199426_f6d48aa585_z.jpg",
        "id": 37777
    },
    ...
]
```

In [ ]:
paths = "/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/samples/sun/HMI_REGION/*.fits"


In [4]:
def images(path):
    tmps = []
    for path in paths:
        tmp = cl.OrderedDict()
        tmp["id"] = 

## 画像データの作成
そもそもCOCOフォーマットのデータを作成する前に画像データ自体を作成しなければならないので指定のフォーマットのデータを作成できるようにする流れは 
- [ ] FITSファイルから画像データの書き出し
- [ ] FITSファイルのメタデータから必要なものを抜きだす


In [5]:
# そもそもの画像の方のデータセットも同時に作る
# from glob import glob
# import sunpy.map
# import cv2
# fits_path = "/media/akito/Data/HMI_REGION/2010/*2010050*"
# paths = sorted(glob(fits_path))
# for i, path in enumerate(paths):
#     # filename="/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/"+path.split(".")[2][0:15]+".jpg"
#     filename="/home/akito/Documents/Documents/Predict_Solar_Flare_Mrcnn/Dataset/Sun/Image/forffmpeg/"+str(i).zfill(3)+".jpg"
#     map=sunpy.map.Map(path)
#     print(filename)
#     cv2.imwrite(filename,cv2.flip(map.data,1))




## 画像のフォーマットについて
Numpyを書き出したら白黒の4k4kのデータになっているがこれで大丈夫そうか？  
→大丈夫そう

## Annotaionの付与について
Pickleの吐き出しまでは良さそうなのでAnnotationをつけるとところをやっていく

In [2]:
# データの読み出し
import utils 
path = "/Users/komatsu/Documents/Predict_Solar_Flare_Mrcnn/coord_df.pickle"
coord_df = utils.pickle_load(path)

In [7]:
# Polygonとそれぞれのフレアデータが正しく入っているか確認
f_len = lambda x : [len(cell) for cell in x]
print(coord_df.apply(f_len))

Polygon  C_FLARE  M_FLARE  X_FLARE
2010-05-01 00:00:00        6        6        6        6
2010-05-01 01:00:00        5        5        5        5
2010-05-01 02:00:00        4        4        4        4
2010-05-01 03:00:00        4        4        4        4
2010-05-01 04:00:00        4        4        4        4
...                      ...      ...      ...      ...
2019-12-31 20:00:00        0        0        0        0
2019-12-31 21:00:00        0        0        0        0
2019-12-31 22:00:00        0        0        0        0
2019-12-31 23:00:00        0        0        0        0
2020-01-01 00:00:00        0        0        0        0

[84769 rows x 4 columns]


In [22]:
import pandas as pd
from shapely.geometry import Polygon
import collections as cl


def annotations(pickle_path):
    tmps = []
    coord_df = pd.read_pickle(pickle_path)
    coord_df.apply(make_annotation_line,tmp=tmps,axis=1)
    # print(coord_df.iloc[0]["Polygon"][0][0][0])
    # return tmps

def make_annotation_line(line,tmp):
    tmp = cl.OrderedDict()
    for i in range(len(line["Polygon"])):
        tmp["segmentation"] = list(itertools.chain.from_iterable(line["Polygon"][i]))
        tmp["id"] = ("{}{}".format(line.name,i))
        polygon = Polygon(line["Polygon"][i])
        bb_coord = polygon.bounds
        # print(polygon.area)
        bbox = [bb_coord[0],bb_coord[1],bb_coord[2]-bb_coord[0],bb_coord[3]-bb_coord[1]]
        # print(bbox)
        # print(line.name)
        
        # TODO:Annotation完成させる

annotations("/home/akito/Documents/Predict_Solar_Flare_Mrcnn/coord_dfs/201005coord_df.pickle")



0.0, 2553.0, 1658.0, 2553.0, 1658.0, 2552.0, 1656.0, 2552.0, 1656.0, 2551.0, 1654.0, 2551.0, 1654.0, 2550.0, 1652.0, 2550.0, 1652.0, 2549.0, 1650.0, 2549.0, 1650.0, 2548.0, 1648.0, 2548.0, 1648.0, 2547.0, 1646.0, 2547.0, 1646.0, 2546.0, 1644.0, 2546.0, 1644.0, 2545.0, 1641.0, 2545.0, 1641.0, 2544.0, 1639.0, 2544.0, 1639.0, 2543.0, 1637.0, 2543.0, 1637.0, 2542.0, 1635.0, 2542.0, 1635.0, 2541.0, 1633.0, 2541.0, 1633.0, 2540.0, 1630.0, 2540.0, 1630.0, 2539.0, 1628.0, 2539.0, 1628.0, 2538.0, 1626.0, 2538.0, 1626.0, 2537.0, 1623.0, 2537.0, 1623.0, 2536.0, 1621.0, 2536.0, 1621.0, 2535.0, 1619.0, 2535.0, 1619.0, 2534.0, 1617.0, 2534.0, 1617.0, 2533.0, 1615.0, 2533.0, 1615.0, 2532.0, 1613.0, 2532.0, 1613.0, 2531.0, 1611.0, 2531.0, 1611.0, 2530.0, 1609.0, 2530.0, 1609.0, 2529.0, 1607.0, 2529.0, 1607.0, 2528.0, 1605.0, 2528.0, 1605.0, 2527.0, 1603.0, 2527.0, 1603.0, 2526.0, 1601.0, 2526.0, 1601.0, 2525.0, 1599.0, 2525.0, 1599.0, 2524.0, 1597.0, 2524.0, 1597.0, 2523.0, 1595.0, 2523.0, 1595.0, 252